## Setup Environment

In [ ]:
! pip install unsloth==2025.4.1 vllm==0.7.3

## Load Base Model

In [ ]:
from vllm import SamplingParams
from unsloth import FastLanguageModel
from trl import GRPOConfig, GRPOTrainer

In [1]:
import os
import re
from datasets import load_dataset, Dataset

d:\Asus\Ungdung\Miniconda\workspace\envs\LLMs_Agents\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configure LoRA

- max_seq_length định nghĩa độ dài tối đa của input sequence.
- lora_rank xác định kích thước của ma trận adapter LoRA.
- load_in_4bit=False tắt lượng tử hóa để giữ độ chính xác cho reasoning.
- fast_inference=True tối ưu hóa throughput khi sinh text.
- gpu_memory_utilization=0.8 cho phép sử dụng tối đa 80% bộ nhớ GPU.

In [ ]:
max_seq_length = 2048
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.2-1B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=False,
    fast_inference=True,
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.8,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=lora_rank,
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

## Prepare Dataset

In [42]:
dataset = load_dataset("5CD-AI/Vietnamese-meta-math-MetaMathQA-40K-gg-translated", split="train")

In [43]:
dataset[3000]

{'query_en': 'John buys 10 packs of magic cards.  Each pack has x cards and 1/4 of those cards are uncommon. He got 50 uncommon cards.',
 'response_en': "To solve this problem, we need to determine the value of x, which represents the number of cards in each pack.\nLet's break down the information given:\nNumber of packs of magic cards: 10\nFraction of uncommon cards in each pack: 1/4\nNumber of uncommon cards obtained: 50\nWe can set up the equation as follows:\nNumber of packs * Number of cards per pack * Fraction of uncommon cards = Number of uncommon cards obtained\n10 * x * 1/4 = 50\nLet's simplify and solve for x:\n10 * x * 1/4 = 50\n10 * x = 50 * 4\n10 * x = 200\nTo isolate x, we divide both sides of the equation by 10:\n10 * x / 10 = 200 / 10\nx = 20\nThe value of x is 20.\n#### 20\nThe answer is: 20",
 'type': 'GSM_SV',
 'query_vi': 'John mua 10 gói thẻ ma thuật. Mỗi gói có x thẻ và 1/4 trong số đó là thẻ không phổ biến. Anh ta có 50 thẻ thông thường.',
 'response_vi': 'Để giả

In [48]:
answer_pattern = re.compile(
    # Sau đó cho phép có khoảng trắng tùy ý (\s*). -- bắt lấy bất kỳ nội dung nào phía sau ((.*)), chính là phần câu trả lời thực sự.
    r"(đáp án là:|đáp án là :|câu trả lời là:|câu trả lời là :)\s*(.*)",
    re.IGNORECASE
)

def extract__qanswer(example):
    # Tách câu hỏi và câu trả lời từ văn bản
    match = answer_pattern.search(example["response_vi"].strip().lower())
    question = example["query_vi"].strip().lower()
    if match:
        answer = match.group(2).strip()
    else:
        answer = ""
    return {"question": question, "answer": answer}


print(extract__qanswer(dataset[0]))

{'question': 'cha của reggie đã cho anh ấy 48 đô la. reggie đã mua 5 cuốn sách, mỗi cuốn có giá x. reggie còn lại 38 tiền. giá trị của biến x chưa biết là bao nhiêu?', 'answer': '2'}


In [54]:
standardized_dataset = dataset.map(
    extract__qanswer,
    remove_columns=dataset.column_names, # xóa các cột hiện tại (vì nó kh cần thiết)
    desc="Standardizing dataset",
).to_list()

In [57]:
standardized_dataset[0]

{'question': 'cha của reggie đã cho anh ấy 48 đô la. reggie đã mua 5 cuốn sách, mỗi cuốn có giá x. reggie còn lại 38 tiền. giá trị của biến x chưa biết là bao nhiêu?',
 'answer': '2'}

In [60]:
reasoning_start = "<thinking>"
reasoning_end = "</thinking>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = \
    f"""You are given a problem.
Think about the problem and provide your thought process.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your final answer between {solution_start}{solution_end}"""

train_dataset = Dataset.from_list(standardized_dataset[:8000]) 

train_dataset = train_dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": x["question"]},
    ],
    "answer": x["answer"],
})


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map: 100%|██████████| 8000/8000 [00:00<00:00, 17551.26 examples/s]


In [64]:
train_dataset[2]

{'question': 'cally và danny giặt quần áo. cally có 10 áo sơ mi trắng, 5 áo sơ mi màu, 7 chiếc quần đùi và 6 chiếc quần dài, trong khi danny có 6 chiếc áo sơ mi trắng, 8 chiếc áo sơ mi màu, 10 chiếc quần đùi và 6 chiếc quần dài. họ đã giặt bao nhiêu bộ quần áo?',
 'answer': '58',
 'prompt': [{'content': 'You are given a problem.\nThink about the problem and provide your thought process.\nPlace it between <thinking> and </thinking>.\nThen, provide your final answer between <SOLUTION></SOLUTION>',
   'role': 'system'},
  {'content': 'cally và danny giặt quần áo. cally có 10 áo sơ mi trắng, 5 áo sơ mi màu, 7 chiếc quần đùi và 6 chiếc quần dài, trong khi danny có 6 chiếc áo sơ mi trắng, 8 chiếc áo sơ mi màu, 10 chiếc quần đùi và 6 chiếc quần dài. họ đã giặt bao nhiêu bộ quần áo?',
   'role': 'user'}]}

## Fine-tune LLM

## Save Checkpoints

## Run Evaluate